In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import random


# =========================
# 1. Load & Utility
# =========================
def load_adbench_data(dataset_path):
    """
    Giả sử file .npz có 'X' và 'y'
    X: (N, d), y: (N,)
    """
    data = np.load(dataset_path)
    X = data['X']
    y = data['y']
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


def evaluate_with_classification_report_and_auc(model, test_loader, device, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).squeeze()  # (B,)
            all_preds.append(y_pred.cpu())
            all_labels.append(y_batch.cpu())

    preds = torch.cat(all_preds).numpy()  # (N_test,)
    labels = torch.cat(all_labels).numpy()  # (N_test,)

    # Chuyển sang nhãn nhị phân
    binary_preds = (preds > threshold).astype(int)

    report = classification_report(labels, binary_preds, target_names=['Class 0', 'Class 1'])
    print(report)

    if len(set(labels)) > 1:
        aucroc = roc_auc_score(labels, preds)
        print(f"AUC-ROC: {aucroc:.4f}")
    else:
        aucroc = None
        print("AUC-ROC: Undefined (only one class present in labels)")
    return report, aucroc


# =========================
# 2. Beta-CVAE
# =========================
class BetaCVAE(nn.Module):
    """
    Beta-CVAE cho bài toán nhị phân:
      - Encoder nhận (x, y)
      - Decoder nhận (z, y)
      - Thêm hệ số beta > 1 cho KL-divergence để khuyến khích latent space 'trải rộng'.
    """

    def __init__(self, input_dim, hidden_dim=128, latent_dim=64, beta=4.0):
        super(BetaCVAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.beta = beta  # hệ số "phóng đại" KL

        # Encoder
        self.fc1 = nn.Linear(input_dim + 1, hidden_dim)  # concat y -> +1
        self.fc2_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)

        # Decoder
        self.fc3 = nn.Linear(latent_dim + 1, hidden_dim)  # concat y -> +1
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x, y):
        xy = torch.cat([x, y], dim=1)  # (B, input_dim+1)
        h = F.relu(self.fc1(xy))
        mean = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z, y):
        zy = torch.cat([z, y], dim=1)  # (B, latent_dim+1)
        h = F.relu(self.fc3(zy))
        x_recon = self.fc4(h)
        return x_recon

    def forward(self, x, y):
        mean, logvar = self.encode(x, y)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decode(z, y)
        return x_recon, mean, logvar


def beta_cvae_loss_fn(x, x_recon, mean, logvar, beta=4.0):
    """
    Reconstruction (MSE) + beta * KL-div
    """
    recon_loss = F.mse_loss(x_recon, x, reduction='sum')
    kl_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return recon_loss + beta * kl_loss


def train_beta_cvae(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device).unsqueeze(1)  # (B,1)

        x_recon, mean, logvar = model(x_batch, y_batch)
        loss = beta_cvae_loss_fn(x_batch, x_recon, mean, logvar, beta=model.beta)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)


# =========================
# 3. Transformer Detector
# =========================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        L = x.size(1)
        return x + self.pe[:, :L, :].to(x.device)


class TransformerDetector(nn.Module):
    def __init__(self, input_size, d_model=128, nhead=8, num_layers=2, dim_feedforward=256, dropout=0.1):
        super(TransformerDetector, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
                                                   nhead=nhead,
                                                   dim_feedforward=dim_feedforward,
                                                   dropout=dropout,
                                                   batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        return self.fc(x).squeeze(1)


def train_detector(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)  # (B,)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)



In [52]:

# =========================
# 4. MAIN DEMO
# =========================
if __name__ == "__main__":
    # Ví dụ đường dẫn
    dataset_path = r"D:\Study\Code\SwiftHydra\Classical\12_fault.npz"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 4.1: Load Data
    X_all, y_all = load_adbench_data(dataset_path)
    input_dim = X_all.shape[1]

    # Chia train/test
    X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
        X_all.numpy(), y_all.numpy(), test_size=0.2, random_state=42, stratify=y_all
    )
    X_train = torch.tensor(X_train_np, dtype=torch.float32)
    y_train = torch.tensor(y_train_np, dtype=torch.float32)
    X_test = torch.tensor(X_test_np, dtype=torch.float32)
    y_test = torch.tensor(y_test_np, dtype=torch.float32)

    # DataLoader cho Beta-CVAE
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # 4.2: Khởi tạo Beta-CVAE
    # Tăng beta (vd=4.0) để nhấn mạnh KL -> latent space "trải rộng" hơn
    beta_cvae = BetaCVAE(input_dim=input_dim, hidden_dim=128, latent_dim=32, beta=1.0).to(device)
    optimizer_cvae = Adam(beta_cvae.parameters(), lr=1e-3)

# Train Beta-CVAE
num_epochs_cvae = 450
for epoch in range(num_epochs_cvae):
    loss_cvae = train_beta_cvae(beta_cvae, train_loader, optimizer_cvae, device)
    if (epoch + 1) % 2 == 0:
        print(f"[Beta-CVAE] Epoch {epoch + 1}/{num_epochs_cvae}, loss={loss_cvae:.2f}")

# 4.3: Sinh dữ liệu - maximize diversity
"""
Thay vì z ~ Normal(0,1), ta thử z ~ Uniform([-2,2]) => 
tăng coverage, khuyến khích dữ liệu sinh ra đa dạng hơn
"""
beta_cvae.eval()

minority_mask = (y_train == 1)
X_minority = X_train[minority_mask]
majority_mask = (y_train == 0)
X_majority = X_train[majority_mask]

num_generate = len(X_majority) - len(X_minority)  # Hoặc tuỳ ý # số lượng sample sinh ra
with torch.no_grad():
    # Tạo z uniform trong [-2,2]
    z_uniform = (torch.rand(num_generate, beta_cvae.latent_dim) * 4.0) - 2.0
    z_uniform = z_uniform.to(device)

    # Gán y=1 => oversample minority
    # hoặc y=0.7 => “lai”
    # Ở đây ta ví dụ oversample minority
    y_synthetic_c = torch.full((num_generate, 1), 0.9, device=device)

    # Decode
    X_synthetic = beta_cvae.decode(z_uniform, y_synthetic_c)
    X_synthetic = X_synthetic.cpu()

# Tạo label = 1 cho samples synthetic
y_synthetic_labels = torch.ones(num_generate)

# Ghép lại
X_train_final = torch.cat([X_train, X_synthetic], dim=0)
y_train_final = torch.cat([y_train, y_synthetic_labels], dim=0)

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

plt.style.use('default')  # Đảm bảo sử dụng style mặc định

# Giả sử bạn đã có:
# - X_train, y_train: Dữ liệu train và nhãn
# - X_test, y_test: Dữ liệu test và nhãn
# - X_synthetic: Dữ liệu synthetic

# 1) Gộp dữ liệu
X_plot = torch.cat([X_train, X_test, X_synthetic], dim=0).numpy()

# Tạo nhãn:
# - Phần đầu: y_train (0 hoặc 1)
# - Tiếp theo: y_test (0 hoặc 1)
# - Cuối cùng: synthetic (2)
N_train = len(X_train)
N_test = len(X_test)
N_synthetic = len(X_synthetic)
y_plot = np.concatenate([
    y_train.numpy(),  # Nhãn train
    y_test.numpy(),  # Nhãn test
    np.full((N_synthetic,), 2)  # Nhãn synthetic
], axis=0)

# 2) Chuẩn hoá dữ liệu (nếu cần)
scaler = StandardScaler()
X_plot_scaled = scaler.fit_transform(X_plot)

# 3) Tính T-SNE
tsne = TSNE(n_components=2, perplexity=50, random_state=42)
X_embedded = tsne.fit_transform(X_plot_scaled)
# X_embedded.shape = (N_train + N_test + N_synthetic, 2)

# 4) Vẽ
plt.figure(figsize=(10, 8), facecolor='white')  # Đặt nền trắng

# Train Class 0 -> đỏ
idx0_train = (y_plot[:N_train] == 0)
plt.scatter(X_embedded[:N_train][idx0_train, 0], X_embedded[:N_train][idx0_train, 1],
            c='darkred', alpha=0.6, label='Class 0 (Train)')

# Train Class 1 -> xanh dương
idx1_train = (y_plot[:N_train] == 1)
plt.scatter(X_embedded[:N_train][idx1_train, 0], X_embedded[:N_train][idx1_train, 1],
            c='darkblue', alpha=0.6, label='Class 1 (Train)')

# Test Class 0 -> cam
idx0_test = (y_plot[N_train:N_train + N_test] == 0)
plt.scatter(X_embedded[N_train:N_train + N_test][idx0_test, 0],
            X_embedded[N_train:N_train + N_test][idx0_test, 1],
            c='orange', alpha=0.6, label='Class 0 (Test)')

# Test Class 1 -> xanh nhạt
idx1_test = (y_plot[N_train:N_train + N_test] == 1)
plt.scatter(X_embedded[N_train:N_train + N_test][idx1_test, 0],
            X_embedded[N_train:N_train + N_test][idx1_test, 1],
            c='skyblue', alpha=0.6, label='Class 1 (Test)')

# Synthetic Data -> xanh lá
idx_syn = (y_plot[N_train + N_test:] == 2)
plt.scatter(X_embedded[N_train + N_test:][idx_syn, 0],
            X_embedded[N_train + N_test:][idx_syn, 1],
            c='green', alpha=0.6, label='Synthetic')

plt.title("T-SNE Visualization: Train, Test, and Synthetic Data")
plt.legend()
plt.show()



[Beta-CVAE] Epoch 2/450, loss=1313.93
[Beta-CVAE] Epoch 4/450, loss=1108.44
[Beta-CVAE] Epoch 6/450, loss=1014.20
[Beta-CVAE] Epoch 8/450, loss=968.30
[Beta-CVAE] Epoch 10/450, loss=936.73
[Beta-CVAE] Epoch 12/450, loss=928.86
[Beta-CVAE] Epoch 14/450, loss=916.00
[Beta-CVAE] Epoch 16/450, loss=892.53
[Beta-CVAE] Epoch 18/450, loss=866.27


KeyboardInterrupt: 

In [ ]:
# 4.4: Train mô hình TransformerDetector
train_dataset_final = TensorDataset(X_train_final, y_train_final)
test_dataset = TensorDataset(X_test, y_test)
train_loader_final = DataLoader(train_dataset_final, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
print("Sau khi oversampling bằng VAE:")
unique, counts = np.unique(y_train_final.numpy(), return_counts=True)
print("Phân phối lớp trong tập train:", dict(zip(unique, counts)))
model = TransformerDetector(input_size=input_dim).to(device)
optimizer_tf = Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()
num_epochs_tf = 100
for epoch in range(num_epochs_tf):
    train_loss = train_detector(model, train_loader_final, optimizer_tf, criterion, device)
    print(f"[Transformer] Epoch {epoch + 1}/{num_epochs_tf}, Loss={train_loss:.4f}")

    # Đánh giá
    print("Test set evaluation:")
    evaluate_with_classification_report_and_auc(model, test_loader, device, threshold=0.5)
    print("-" * 40)

In [ ]:
import os

# Đảm bảo thư mục lưu trữ tồn tại
save_dir = "./saved_models"
os.makedirs(save_dir, exist_ok=True)

# Lưu Beta-CVAE
vae_path = os.path.join(save_dir, "beta_cvae.pth")
torch.save(beta_cvae.state_dict(), vae_path)
print(f"Beta-CVAE model saved to: {vae_path}")

# Lưu TransformerDetector
detector_path = os.path.join(save_dir, "transformer_detector.pth")
torch.save(model.state_dict(), detector_path)
print(f"TransformerDetector model saved to: {detector_path}")


In [53]:
# Khởi tạo mô hình cùng cấu hình ban đầu
loaded_beta_cvae = BetaCVAE(input_dim=input_dim, hidden_dim=128, latent_dim=32, beta=1.0).to(device)
loaded_detector_model = TransformerDetector(input_size=input_dim).to(device)

# Load trạng thái mô hình đã lưu
loaded_beta_cvae.load_state_dict(torch.load(vae_path))
loaded_detector_model.load_state_dict(torch.load(detector_path))

# Đặt mô hình về chế độ eval (nếu chỉ sử dụng inference)
loaded_beta_cvae.eval()
loaded_detector_model.eval()

print("Models loaded successfully.")


Models loaded successfully.


C:\Users\Legion 7\AppData\Local\Temp\ipykernel_29568\1091737754.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_beta_cvae.load_state_dict(torch.load(vae_path))
C:

In [54]:
num_episodes = 100

num_gen_data = 20

In [98]:
import os
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# =========================
# 1. Hàm tiện ích
# =========================
def log_to_file(file_path, message):
    """
    Ghi message vào file log cụ thể.
    """
    with open(file_path, "a") as file:
        file.write(message + "\n")

# =========================
# 2. Hàm train và đánh giá
# =========================
def train_beta_cvae(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device).unsqueeze(1)

        x_recon, mean, logvar = model(x_batch, y_batch)
        recon_loss = nn.MSELoss(reduction="sum")(x_recon, x_batch)
        kl_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
        loss = recon_loss + model.beta * kl_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def train_detector(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_detector(model, test_loader, device, threshold=0.5, log_file=None):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = (outputs > threshold).float()
            all_labels.extend(y_batch.cpu().tolist())
            all_preds.extend(preds.cpu().tolist())
    report = classification_report(all_labels, all_preds, digits=4)
    if log_file:
        log_to_file(log_file, report)
    print(report)

# =========================
# 3. Sinh adversarial sample
# =========================
def generate_one_adversarial_sample(
    beta_cvae, 
    detector, 
    x_orig, 
    device,
    previously_generated=None, 
    alpha=1.0, 
    lambda_div=0.1,
    lr=0.001, 
    steps=50,
    log_file=None
):
    beta_cvae.eval()
    detector.eval()
    if previously_generated is None:
        previously_generated = []

    x_orig = x_orig.to(device).unsqueeze(0)
    y_class1 = torch.ones((1, 1), device=device)

    with torch.no_grad():
        mean, logvar = beta_cvae.encode(x_orig, y_class1)
        z = beta_cvae.reparameterize(mean, logvar).detach().clone().requires_grad_(True)

    optimizer_z = torch.optim.Adam([z], lr=lr)
    for step in range(steps):
        optimizer_z.zero_grad()
        x_synthetic = beta_cvae.decode(z, y_class1)
        prob_class1 = detector(x_synthetic)
        if previously_generated :  # chỉ tính nếu không rỗng
            # 1) Gộp lại thành tensor (N, d)
            x_old_cat = torch.stack(previously_generated, dim=0).to(device)  # (N, d)
        
            # 2) Tính norm theo batch
            # Cách 1: Sử dụng broadcast (x_synthetic - x_old_cat)
            # dist shape (N,)
            dist = torch.norm(x_synthetic - x_old_cat, p=2, dim=1)
            
            # 3) Tính sum của exp(-alpha * dist)
            diversity_term = torch.exp(-alpha * dist).sum()
        
        else:
            diversity_term = 0.0

        loss = prob_class1.mean() + lambda_div * diversity_term
        loss.backward()
        optimizer_z.step()

        if log_file:
            log_to_file(log_file, f"Step {step + 1}/{steps}:")
            log_to_file(log_file, f"  Prob_class1: {prob_class1.item():.4f}")
            log_to_file(log_file, f"  Diversity term: {diversity_term:.4f}")
            log_to_file(log_file, f"  Total loss: {loss.item():.4f}")
        # print(f"Step {step + 1}/{steps}:")
        # print(f"  Prob_class1: {prob_class1.item():.4f}")
        # print(f"  Diversity term: {diversity_term:.4f}")
        # print(f"  Total loss: {loss.item():.4f}")

    with torch.no_grad():
        x_adv = beta_cvae.decode(z, y_class1).detach().cpu().squeeze(0)
    # previously_generated.append(x_adv)
    return x_adv

# =========================
# 4. MAIN DEMO
# =========================
if __name__ == "__main__":
    # File log
    log_dir = "./logs"
    os.makedirs(log_dir, exist_ok=True)
    beta_cvae_log = os.path.join(log_dir, "beta_cvae.log")
    detector_log = os.path.join(log_dir, "detector.log")
    adversarial_log = os.path.join(log_dir, "adversarial_samples.log")

    # Dữ liệu
    dataset_path = r"D:\Study\Code\SwiftHydra\Classical\12_fault.npz"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X_all, y_all = load_adbench_data(dataset_path)
    X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
        X_all.numpy(), y_all.numpy(), test_size=0.2, random_state=42, stratify=y_all
    )
    X_train = torch.tensor(X_train_np, dtype=torch.float32)
    y_train = torch.tensor(y_train_np, dtype=torch.float32)
    X_test = torch.tensor(X_test_np, dtype=torch.float32)
    y_test = torch.tensor(y_test_np, dtype=torch.float32)
    
        # Lấy mask cho class 1
    class1_mask = (y_train == 1)  # Boolean tensor: True nếu y_train[i] == 1
    
    # Lọc các mẫu thuộc class 1 từ X_train
    # Lọc các mẫu thuộc class 1 từ X_train
    X_train_no_grow_tensor = X_train[class1_mask]  # Tensor chứa tất cả các mẫu class 1
    
    # Chuyển mỗi hàng thành một tensor và lưu vào list
    X_train_no_grow = [row for row in X_train_no_grow_tensor]

    
    # print(X_train_no_grow)

    # Khởi tạo mô hình
    input_dim = X_train.shape[1]
    beta_cvae = BetaCVAE(input_dim=input_dim, hidden_dim=128, latent_dim=32, beta=1.0).to(device)
    new_detector = TransformerDetector(input_size=input_dim).to(device)
    optimizer_cvae = Adam(beta_cvae.parameters(), lr=1e-4)
    optimizer_detector = Adam(new_detector.parameters(), lr=1e-4)
    criterion = nn.BCELoss()

    # Train Beta-CVAE
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    num_epochs_cvae = 50
    for epoch in range(num_epochs_cvae):
        loss_cvae = train_beta_cvae(loaded_beta_cvae, train_loader, optimizer_cvae, device)
        log_to_file(beta_cvae_log, f"Epoch {epoch + 1}/{num_epochs_cvae}, Loss: {loss_cvae:.4f}")

    # Train Detector với Adversarial Samples
    num_episodes = 100
    num_gen_data = 10
    batch_size = 64
    previously_generated = []
    for ep in range(num_episodes):
        print(f"===== EPISODE {ep + 1}/{num_episodes} =====")
        # Train Detector
        detector_epochs = 10
        train_dataset_detector = TensorDataset(X_train, y_train)
        train_loader_detector = DataLoader(train_dataset_detector, batch_size=batch_size, shuffle=True)
        for det_epoch in range(detector_epochs):
            detector_loss = train_detector(new_detector, train_loader_detector, optimizer_detector, criterion, device)
            log_to_file(detector_log, f"Epoch {det_epoch + 1}/{detector_epochs}, Loss: {detector_loss:.4f}")
        
        # print(f"===== Evaluate in Training set =====")
        # evaluate_detector(new_detector, DataLoader(TensorDataset(X_train, y_train)), device, log_file=detector_log)
        
        # Giả sử y_train có dạng 0/1
        unique_labels, counts = torch.unique(y_train, return_counts=True)
        for lbl, cnt in zip(unique_labels, counts):
            print(f"Class {lbl.item()}: {cnt.item()} samples")
        print(f"===== Evaluate in Testing set =====")
        evaluate_detector(new_detector, DataLoader(TensorDataset(X_test, y_test)), device, log_file=detector_log)

        # Generate Adversarial Samples
        idx_class1 = (y_train == 1).nonzero(as_tuple=True)[0]
        new_samples, new_labels = [], []
        for _ in range(num_gen_data):
            random_idx = random.choice(idx_class1)
            x_orig = X_train[random_idx]
            x_adv = generate_one_adversarial_sample(
                beta_cvae=beta_cvae,
                detector=loaded_detector_model,
                x_orig=x_orig,
                device=device,
                previously_generated=X_train_no_grow,
                alpha=1.0,
                lambda_div=0.1,
                lr=0.01,
                steps=50,
                log_file=adversarial_log,
            )
            new_samples.append(x_adv.unsqueeze(0))
            new_labels.append(torch.tensor([1]))
        new_samples = torch.cat(new_samples, dim=0)
        new_labels = torch.cat(new_labels, dim=0)
        X_train = torch.cat([X_train, new_samples], dim=0)
        y_train = torch.cat([y_train, new_labels], dim=0)

===== EPISODE 1/100 =====
Class 0.0 => 1014 samples
Class 1.0 => 538 samples
===== Evaluate in Testing set =====
              precision    recall  f1-score   support

         0.0     0.7705    0.9252    0.8408       254
         1.0     0.7738    0.4815    0.5936       135

    accuracy                         0.7712       389
   macro avg     0.7722    0.7033    0.7172       389
weighted avg     0.7716    0.7712    0.7550       389

===== EPISODE 2/100 =====
Class 0.0 => 1014 samples
Class 1.0 => 548 samples
===== Evaluate in Testing set =====
              precision    recall  f1-score   support

         0.0     0.7835    0.8976    0.8367       254
         1.0     0.7347    0.5333    0.6180       135

    accuracy                         0.7712       389
   macro avg     0.7591    0.7155    0.7274       389
weighted avg     0.7666    0.7712    0.7608       389

===== EPISODE 3/100 =====
Class 0.0 => 1014 samples
Class 1.0 => 558 samples
===== Evaluate in Testing set =====
       

KeyboardInterrupt: 

Initial TRAIN distribution:
  Class 0.0: 1014 samples
  Class 1.0: 538 samples
Loaded pretrained Beta-CVAE and Detector successfully.


C:\Users\Legion 7\AppData\Local\Temp\ipykernel_29568\2928958269.py:144: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_beta_cvae.load_state_dict(torch.load(vae_path))


Fine-tuned Beta-CVAE for 10 epochs.

=== EPISODE 1/100 ===
  [Detector] final train loss = 0.0283
  Evaluate on TEST set:
              precision    recall  f1-score   support

         0.0     0.8293    0.8031    0.8160       254
         1.0     0.6503    0.6889    0.6691       135

    accuracy                         0.7635       389
   macro avg     0.7398    0.7460    0.7425       389
weighted avg     0.7672    0.7635    0.7650       389

  Current distribution => class0.0: 1014, class1.0: 538
  => Generating 10 new samples for class 1

=== EPISODE 2/100 ===
  [Detector] final train loss = 0.0223
  Evaluate on TEST set:
              precision    recall  f1-score   support

         0.0     0.8185    0.7992    0.8088       254
         1.0     0.6383    0.6667    0.6522       135

    accuracy                         0.7532       389
   macro avg     0.7284    0.7329    0.7305       389
weighted avg     0.7560    0.7532    0.7544       389

  Current distribution => class0.0: 101

KeyboardInterrupt: 